# Content and Objective

+ Showing effects of varying length of impulse response

+ White gaussian noise is sampled and filtered with impulse response constisting of "1"!
$$X[n] \sim\mathcal{N}(0, 1), \quad n=0, \ldots, N_\text{signal}$$
$$ h[n] = \begin{cases} \frac{1}{\sqrt{ N_\text{filter} }}, & 0\leq n\leq N_\text{filter} - 1 \\ 0 , & \text{otherwise}\end{cases} $$
$$Y[n]=X[n]\ast h[n], \quad 0\leq n\leq N_\text{signal}+N_\text{filter} - 1$$ 

# Importing

In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from matplotlib.animation import PillowWriter # Disable if you don't want to save any GIFs.

from ipywidgets import interactive
import ipywidgets as widgets
%matplotlib inline 

# Misc Helper Function

In [2]:
# function for applying filtering operation
# NOTE: function has option "normalized" to not increase output power w. filter length
def apply_filter( x: np.ndarray, filter_length: int, normalized: int = 1 ) -> np.ndarray :

    # init with according amount of ones
    h = np.ones( filter_length )
    
    # want to normalize?
    if normalized:
        h /= np.sqrt( filter_length )

    return np.convolve( x, h, 'full' )


# function for getting acf
def get_acf( x: np.ndarray, is_unbiased = 0 ) -> np.ndarray :

    if is_unbiased:
        factor = np.linspace( -x.size, x.size, 2 * x.size - 1 )
    else:
        factor = x.size

    return np.correlate( x, x, 'full' ) / factor


# function for getting psd
def get_psd( x: np.ndarray ) -> np.ndarray :

    #return 1 / x.size * np.abs( np.fft.fft( get_acf( x ) ) )**2
    return np.fft.fft( get_acf( x ) )

# Interactive Plots

### Show signals

In [3]:
# sampling time
t_sample = 1

# get length and  samples of input signal
signal_length = 1000
x = np.random.randn( signal_length )


# time vector of x
in_time = np.linspace( 0, signal_length, x.size ) * t_sample


# interactive plotting function
def plot_filter_output( filter_length ):

    # get output signal by filtering 
    y = apply_filter( x, filter_length )
    
    # number of samples at output
    out_time = np.arange( 0, signal_length + filter_length - 1 ) * t_sample


    # figure for time signals
    plt.figure( figsize=( 20, 8 ) )
    font = {'size'   : 14}
    plt.rc('font', **font)
    plt.rc('text', usetex=matplotlib.checkdep_usetex(True))
    plt.ylim( (np.min(x), np.max(x)))

    plt.plot( in_time, x, label='$x[n]$' )
    plt.plot( out_time, y, label='$y[n]$'  )

    plt.xlabel(r'$n$',fontsize=14)
    plt.legend( )
    plt.grid( True )
    plt.title('Signals')    


interactive_update = interactive( 
    plot_filter_output, 
    filter_length = widgets.IntSlider(
            min = 1, max = signal_length, step = 1, value = 1, 
            continuous_update = False, 
            description = 'Filter length', 
            style={'description_width': 'initial'}, 
            layout=widgets.Layout(width='50%'),
            align_items='center',
            )
    )


output = interactive_update.children[-1]
output.layout.height = '500px'
interactive_update

interactive(children=(IntSlider(value=1, continuous_update=False, description='Filter length', layout=Layout(w…

### Show acf

In [4]:
# get acf
acf_x = get_acf( x ) 


# time vector of acf of x
in_time_acf = np.linspace( -x.size, x.size, acf_x.size ) 


# interactive plotting function
def plot_filter_output( filter_length ):

   # apply filter and get acf
    y = apply_filter( x, filter_length )
   
    acf_y = get_acf( y )
    out_time_acf = np.linspace( -y.size, y.size, acf_y.size ) 


    # figure for acfs
    plt.figure( figsize = (18,8) ) 
    font = {'size'   : 14}
    plt.rc('font', **font)
    plt.rc('text', usetex=matplotlib.checkdep_usetex(True))
    

    plt.plot( in_time_acf, acf_x, label='$\\varphi_{XX}[k]$' )
    plt.plot( out_time_acf, acf_y, label='$\\varphi_{YY}[k]$' )

    plt.xlabel(r'$k$',fontsize=14)
    plt.legend( )
    plt.grid( True )
    plt.title('ACFs')    


interactive_update = interactive( 
    plot_filter_output, 
    filter_length = widgets.IntSlider(
            min = 1, max = signal_length, step = 1, value = 1, 
            continuous_update = False, 
            description = 'Filter length', 
            style={'description_width': 'initial'}, 
            layout=widgets.Layout(width='50%'),
            align_items='center',
            )
    )

output = interactive_update.children[-1]
output.layout.height = '500px'
interactive_update

interactive(children=(IntSlider(value=1, continuous_update=False, description='Filter length', layout=Layout(w…